In [37]:
from transformers import BertTokenizer, BertModel, BertForTokenClassification
from sklearn.metrics import f1_score
import torch
import json
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
with open('/kaggle/input/train-json/Subtask_1_train.json', 'r') as train_file:
    train_data = json.load(train_file)

for i in train_data:
    print(i)
    break

{'conversation_ID': 1, 'conversation': [{'utterance_ID': 1, 'text': 'Alright , so I am back in high school , I am standing in the middle of the cafeteria , and I realize I am totally naked .', 'speaker': 'Chandler', 'emotion': 'neutral'}, {'utterance_ID': 2, 'text': 'Oh , yeah . Had that dream .', 'speaker': 'All', 'emotion': 'neutral'}, {'utterance_ID': 3, 'text': 'Then I look down , and I realize there is a phone ... there .', 'speaker': 'Chandler', 'emotion': 'surprise'}, {'utterance_ID': 4, 'text': 'Instead of ... ?', 'speaker': 'Joey', 'emotion': 'surprise'}, {'utterance_ID': 5, 'text': 'That is right .', 'speaker': 'Chandler', 'emotion': 'anger'}, {'utterance_ID': 6, 'text': 'Never had that dream .', 'speaker': 'Joey', 'emotion': 'neutral'}, {'utterance_ID': 7, 'text': 'No .', 'speaker': 'Phoebe', 'emotion': 'neutral'}, {'utterance_ID': 8, 'text': 'All of a sudden , the phone starts to ring .', 'speaker': 'Chandler', 'emotion': 'neutral'}], 'emotion-cause_pairs': [['3_surprise', 

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

temp_texts = [item['conversation'] for item in train_data]
temp_labels = [item['conversation'] for item in train_data]

train_texts = []
train_labels = []

for x in temp_texts:
    temp_t = []
    temp_l = []
    for y in x:
        temp_t.append(y['text'])
        temp_l.append(y['emotion'])
    train_texts.append(temp_t)
    train_labels.append(temp_l)

['Alright , so I am back in high school , I am standing in the middle of the cafeteria , and I realize I am totally naked .', 'Oh , yeah . Had that dream .', 'Then I look down , and I realize there is a phone ... there .', 'Instead of ... ?', 'That is right .', 'Never had that dream .', 'No .', 'All of a sudden , the phone starts to ring .']
['neutral', 'neutral', 'surprise', 'surprise', 'anger', 'neutral', 'neutral', 'neutral']


In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
bert_model = BertModel.from_pretrained('bert-base-cased')
bert_model = bert_model.to(device)

In [5]:
labels_dict = {'anger'  :1,
               'joy'    :2,
               'fear'   :3,
               'disgust':4,
               'neutral':5,
               'surprise':6,
               'sadness':7
              }

inverse_dict = {}
for i in labels_dict:
    inverse_dict[labels_dict[i]] = i
inverse_dict[0] = 'neutral'

In [6]:
train_text = train_texts[:1100]
val_text = train_texts[1100:]
train_label = train_labels[:1100]
val_label = train_labels[1100:]
print(len(train_text))
print(len(val_text))

1100
274


In [15]:
for i in val_text:
    print(i)
    break

for i in val_label:
    print(i)
    break

['I can not believe this ! How long as this been going on ?', 'Well umm , Alexandra has been', 'Oh , it has a name ?', 'Phoebe , do not get upset !', 'Okay ... Oop ! Too late ! I am leaving ! Come on Chandler let go !', 'Well , Phoebe I thought I would ... Yeah , what the hell .', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
['surprise', 'neutral', 'surprise', 'sadness', 'anger', 'neutral']


In [11]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.lstm1 = nn.LSTM(hidden_size, hidden_size//2)
        self.fc2 = nn.Linear(hidden_size//2, hidden_size//4)
        self.fc3 = nn.Linear(hidden_size//4, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out, _ = self.lstm1(out)
        out = self.fc2(out)
        out = torch.relu(out)
        out = self.fc3(out)
        return out

input_size = 768
hidden_size = 512
output_size = 8

lstm_model = LSTMModel(input_size, hidden_size, output_size)
lstm_model = lstm_model.to(device)
state_dict = torch.load('/kaggle/input/nlp_project_model/pytorch/models/1/lstm.pth')
lstm_model.load_state_dict(state_dict)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm_model.parameters(), lr=0.001)

In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
bert_model = BertModel.from_pretrained('bert-base-cased')
bert_model = bert_model.to(device)

In [21]:
sentence_list = []
generated_emotion_list = []

In [22]:
for xx in val_text:
    lstm_model.eval()
    text = xx
    temp_len = len(text)
    if(temp_len<35):
        for lol in range(35-temp_len):
            text.append("")

    inputs = tokenizer(text, return_tensors="pt", padding='max_length', truncation=True, max_length=512)
    inputs = {key: val.to(device) for key, val in inputs.items()}
    with torch.no_grad():
        outputs = bert_model(**inputs)

    output = lstm_model(outputs[0].mean(dim=1))
    final_result = []
    for i in range(temp_len):
        x = output[i]
        probabilities = torch.nn.functional.softmax(x, dim=0)
        predicted_class = torch.argmax(probabilities).item()
        final_result.append(inverse_dict[predicted_class])
    removed_text = []
    removed_emotion = []
    for i, j in enumerate(text):
        if(j!=""):
            removed_text.append(text[i])
            removed_emotion.append(final_result[i])
    sentence_list.append(removed_text)
    generated_emotion_list.append(removed_emotion)

In [35]:
import pickle
with open('sentence_list.txt', 'wb') as file:
    pickle.dump(sentence_list, file)
with open('generated_emotion_list.txt', 'wb') as file:
    pickle.dump(generated_emotion_list, file)

In [61]:
generated_text_list = []
for oo in range(len(sentence_list)):
    text = sentence_list[oo]
    emotions = generated_emotion_list[oo]
    emotion_idx = []
    generated_text = []
    for i, temotion in enumerate(emotions):
        if temotion!="neutral":
            emotion_idx.append(i)
            temp_l = []
            temp_l.append(temotion)
            temp_l.append(":")
            for j in range(i+1):
                temp_l.extend(text[j].split())
            generated_text.append(temp_l)
    generated_text_list.append([text,emotions,generated_text,emotion_idx])
print(len(generated_text_list))

274


In [62]:
print(generated_text_list[0])

[['I can not believe this ! How long as this been going on ?', 'Well umm , Alexandra has been', 'Oh , it has a name ?', 'Phoebe , do not get upset !', 'Okay ... Oop ! Too late ! I am leaving ! Come on Chandler let go !', 'Well , Phoebe I thought I would ... Yeah , what the hell .'], ['surprise', 'neutral', 'surprise', 'anger', 'anger', 'neutral'], [['surprise', ':', 'I', 'can', 'not', 'believe', 'this', '!', 'How', 'long', 'as', 'this', 'been', 'going', 'on', '?'], ['surprise', ':', 'I', 'can', 'not', 'believe', 'this', '!', 'How', 'long', 'as', 'this', 'been', 'going', 'on', '?', 'Well', 'umm', ',', 'Alexandra', 'has', 'been', 'Oh', ',', 'it', 'has', 'a', 'name', '?'], ['anger', ':', 'I', 'can', 'not', 'believe', 'this', '!', 'How', 'long', 'as', 'this', 'been', 'going', 'on', '?', 'Well', 'umm', ',', 'Alexandra', 'has', 'been', 'Oh', ',', 'it', 'has', 'a', 'name', '?', 'Phoebe', ',', 'do', 'not', 'get', 'upset', '!'], ['anger', ':', 'I', 'can', 'not', 'believe', 'this', '!', 'How', '

In [38]:
classifier_model = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=2).to(device)
state_dict = torch.load('/kaggle/input/nlp_project_model/pytorch/models/1/overall_model.pth')
classifier_model.load_state_dict(state_dict)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [42]:
def text_tokenizer(text):
    text_str = ""   
    for k in text:
        text_str+=k
        text_str+=" "
    inputs = tokenizer(text_str, return_tensors="pt", padding='max_length', truncation=True, max_length=512)
    return inputs

In [66]:
all_items = generated_text_list[0][2]
predicted_together = []

for items in all_items:
    text_list = items
    inputs = text_tokenizer(text_list)
    inputs = inputs.to(device)
    with torch.no_grad():
        outputs = classifier_model(**inputs)
        predicted = outputs.logits.argmax(dim=-1)
    final_predicted = predicted.flatten().tolist()[:len(text_list)]
    predicted_together.append(final_predicted)

In [74]:
print(predicted_together)
print(generated_text_list[0])
text = generated_text_list[0][0]
emotion = generated_text_list[0][1]
splitted_text = generated_text_list[0][2]
emotion_idx = generated_text_list[0][3]

[[1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]]
[['I can not believe this ! How long as this been going on ?', 'Well umm , Alexandra has been', 'Oh , it has a name ?', 'Phoebe , do not get upset !', 'Okay ... Oop ! Too late ! I am leaving ! Come on Chandler let go !', 'Well , Phoebe I thought I would ... Yeah , what the hell .'], ['surprise', 'neutral', 'surprise', 'anger', 'anger', 'neutral'], [['surprise', ':', 'I', 'can', 'not', 'believe', 'this', '!', 'How', 'long', 'as', 'this', 'been', 'going', 'on', '?'], ['surprise', ':', 'I', 'can', 'not', 'believe', 'this', '!', 'How', 'long', 'as', 'this', 'been', 'going', 'on', '?', '

In [91]:
for i, j in enumerate(emotion_idx):
    upper_text = str(j+1)+"_"+emotions[j]
    down_text = ""
    for idx, x in enumerate(predicted_together[i][2:]):
        if(x==1):
            down_text+=splitted_text[i][2:][idx]+" "
    down_text = down_text[:len(down_text)-1]
    for idx, x in enumerate(text):
        if down_text in x:
            down_text = str(idx+1)+"_"+down_text
            break
    print(upper_text)
    print(down_text)

1_surprise
1_I can not believe this ! How long as this been going on ?
3_surprise
2_Alexandra has been
4_anger
Alexandra get upset !
5_anger
5_late ! I am leaving ! Come
